# Neural networks for credit risk evaluation: Investigation of different neural models and learning schemes 
## Author: Sikai Feng 

## Introduction
In this jupiter notebook, we are going to work on the application of Machine Learning in risk management. Most part of the work is based on the paper 'Neural networks for credit risk evaluation: Investigation of different neural models and learning schemes' written by Adnan Khashman.

The Machine Learning model we apply here is supervised neural network models based on the back propagation learning algorithm. In the original paper, the author trained three neural networks to decide whether to approve or reject a credit application. The neural networks are trained using real world credit application cases from the German credit approval datasets which has 1000 cases; each case with 24 numerical attributes; based on which an application is accepted or rejected. Nine learning schemes with different training-to-validation data ratios have been investigated, and a comparison between their implementation results has been provided. Experimental results will suggest which neural network model, and under which learning scheme, can the proposed credit risk evaluation system deliver optimum performance;where it may be used efficiently, and quickly in automatic processing of credit applications.

However, since some of the methodology the paper applied on the model fitting could be problematic, except for implementing what the paper did, we will also explore potential improvement methods to tune the model, increasing reliability and predicting power.


## Problem to solve
In the paper of Adnan Khashman, the author mainly solved 3 problems, which are the first 3 problems I listed below. We are also going to focus on that. However, in addition, we need to anwswer whether the model is free of flaw, and if yes, then we test whether the flaw contained in the model can substantialy undermine the reliability of the model, and the solution to fix it.

1)The first problem when using neural networks is the use of a high ratio of training-to-validation datasets. Though varying from different dataset, a high ratio of training-to-validation data does not yield meaningful learning

2)The second problem is quantification and normalization of data. Since our original data (German Credit Data) come from questionary banks give to their clients, so it contains both numerical and categorical data. We should transfer categorical data into numerical data. And then we should normalize the data. Since some attributes might be too large and they may influence other attribute to be fairly evaluated in the model.

3)The third problem is computational cost. Traditionaly, most credit evaluation neural network model uses three hidden layer. However, with more layer used in the model, computational cost will be higher , and thus, processing time becomes higher as well. We also need to include pridicting time, since it is a direct measure how fast bank can process credit application.

4)The fourth problem is model tuning. We will verify the reliability of some choices of model parameters, for example, the choice of solver , the choce of learning rate and the number of hidden layer included. We need to compare the change of loss value and prodicting power after selecting a different parameter, hopefully we may improve the model mentioned in the paper.

Now, let's begin work on the data and solve above problems.

## Import dataset

In [62]:
#Import Library that will be used in the neural network classifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from datetime import datetime as dt
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import mean_squared_error as mse
%matplotlib inline

The data we will work on, German Credit Data, chosen by the paper, can be downloaded from UCI Machine Learning data repository, https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/. It is a real world dataset,which classifies credit applicants described by a set of attributes as good or bad credit risks. 

The German Credit Data contains 1000 instances or cases of loan applications. The original data has a mix of 20 categorical and numerical attributes. To make best use of the data, we need to transform categorical data into numerical data. The choice of transforming data can be a tricky one, since we must retain the information contained in the original numerical data, and some logical connections among cotegorical data are hard to comprehend after transform. Fortunately, for today, we do not have to worry too much on this problem, because, in the repository, a numeric version of this dataset is also available, where the categorical attributes are transformed into numerical ones and a few indicator variables are added, which increases the dimension to 24 input numerical values. The data instances are labeled as classes 1 (good, 700 instances) and 2 (bad, 300 instances). 

We will choose numerical dataset after categorical transforming to run our model. But since Neural Network Classifier's target value has to be 0 or 1, one more thing we should do is deduce original target value by 1 to meet this need.

In [63]:
df=pd.read_csv("german.data-numeric.csv")# import numerical data
df['Target']=df['Target']-1 #transfer targer value to binary

In [64]:
df.head() #show heads of the original data.

,Attribute 1,Attribute 2,Attribute 3,Attribute 4,Attribute 5,Attribute 6,Attribute 7,Attribute 8,Attribute 9,Attribute 10,...,Attribute 16,Attribute 17,Attribute 18,Attribute 19,Attribute 20,Attribute 21,Attribute 22,Attribute 23,Attribute 24,Target
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,0
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,1
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,0
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,0
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,1


## Normalization Data
In the paper,the author apply a simple ormalization procedure, which is, for each attribute, the author divided each value in the column by the largest recorded value in that column, and since all of our data are positive, all the dataset attributes obtained will be between 0 and 1.

However, this method conducted by the paper could be problematic. The reason we want to normalize data is because when running Neural Network model, if some attributes has a value that is too large, it will impact the contribution of other contribute to the model, therefore undermine the credibility of the preciting power of the model. The above method Adnan applied did ensure the achievement of previous goal, but it also neglect the fact that scaling will cause different variances among attributes. An ideal way of normalization method we should choose is by subtracting the mean of each attribute and dividing variance, then we will obtain data with mean 0 and variance 1.

We will see how two normalization methods differentiate with each other when producing result.

In [65]:
#normalize the data in the way paper suggested
max_value=[df[i].max() for i in df.columns]#Find the largest value in each column
for i in df.columns:
    df1=df.apply(lambda x:x/x.max())#For every column, divide every number by the largest value in each column
df1.head()

,Attribute 1,Attribute 2,Attribute 3,Attribute 4,Attribute 5,Attribute 6,Attribute 7,Attribute 8,Attribute 9,Attribute 10,...,Attribute 16,Attribute 17,Attribute 18,Attribute 19,Attribute 20,Attribute 21,Attribute 22,Attribute 23,Attribute 24,Target
0,0.25,0.083333,1.00,0.065217,1.0,1.0,0.75,1.00,0.25,0.893333,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.50,0.666667,0.50,0.326087,0.2,0.6,0.50,0.50,0.25,0.293333,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,1.00,0.166667,1.00,0.114130,0.2,0.8,0.75,0.75,0.25,0.653333,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.25,0.583333,0.50,0.429348,0.2,0.8,0.75,1.00,0.50,0.600000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.25,0.333333,0.75,0.266304,0.2,0.6,0.75,1.00,1.00,0.706667,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [66]:
#normalize the data by convert it into normal distribution
for i in df.columns:
    df2[['Attribute '+str(i) for i in range(1,25)]]=df[['Attribute '+str(i) for i in range(1,25)]].apply(lambda x:(x-x.mean())/x.std())#Substracting mean and divide by variance for each record in one attribute
df2['Target']=df['Target']
df2.head()

,Attribute 1,Attribute 2,Attribute 3,Attribute 4,Attribute 5,Attribute 6,Attribute 7,Attribute 8,Attribute 9,Attribute 10,...,Attribute 16,Attribute 17,Attribute 18,Attribute 19,Attribute 20,Attribute 21,Attribute 22,Attribute 23,Attribute 24,Target
0,-1.253938,-1.235859,1.343342,-0.733065,1.832252,1.337409,0.449102,1.046463,-1.293076,2.765073,...,-0.552429,-0.338692,0.320052,-0.206664,-0.4667,0.634131,-0.149908,-0.49975,0.765973,0
1,-0.458797,2.247070,-0.503176,0.965893,-0.699357,-0.317800,-0.963168,-0.765594,-1.293076,-1.190808,...,-0.552429,-0.338692,0.320052,-0.206664,-0.4667,0.634131,-0.149908,-0.49975,0.765973,1
2,1.131486,-0.738298,1.343342,-0.414510,-0.699357,0.509805,0.449102,0.140434,-1.293076,1.182721,...,-0.552429,-0.338692,0.320052,-0.206664,-0.4667,0.634131,-0.149908,1.99900,-1.304224,0
3,-1.253938,1.749509,-0.503176,1.638398,-0.699357,0.509805,0.449102,1.046463,-0.340885,0.831087,...,-0.552429,-0.338692,-3.121368,-0.206664,-0.4667,-1.575384,-0.149908,-0.49975,0.765973,0
4,-1.253938,0.256825,0.420083,0.576549,-0.699357,-0.317800,0.449102,1.046463,1.563498,1.534354,...,1.808378,-0.338692,0.320052,-0.206664,-0.4667,-1.575384,-0.149908,-0.49975,0.765973,1


From above two table, we noticed the range of data varies after applying two different normalization methods. Clearly the values in second one are in a larger range. 

# Evaluation system
In the paper, Adnan trained three neural network models using nine learning schemes on German Credit Data. The three neural models differ in topology, and in particular in the number of hidden layer neurons and learning and momentum rates. The nine learning schemes differ in the training-to-validation data ratios (or as we refer to them, learning ratios). The lower the ratio, the more challenging it is for a neural network, but the more robust and meaningful the learning is. He compared the performance of the neural network models under all schemes and then selected the ideal neural model and learning scheme.

We will reproduce what Adnan did, and work further, trying different solver, hidden layer and learning_rate, and evaluating their impact on the model.

## Define the scheme
We are trying to use different scheme to find the optimal training-to-validation ratios. The the 9 schemes we defined here have 9 different training-to-validation ratios, including 1:9, 2:8,...,9:1. We will use sample function of DataFrame to select a number of training data, and use the rest as validation data.

In [67]:
#Nine different learning schemes in the first normalization method.
LS1_Training_=df1.sample(100) #The training data contains 100 data points.
LS1_Testing_=df1[df1.index.isin(LS1_Training.index)==False] #The validation data contains the other 900 data points.
LS2_Training_=df1.sample(200)
LS2_Testing_=df1[df1.index.isin(LS2_Training.index)==False]
LS3_Training_=df1.sample(300)
LS3_Testing_=df1[df1.index.isin(LS3_Training.index)==False]
LS4_Training_=df1.sample(400)
LS4_Testing_=df1[df1.index.isin(LS4_Training.index)==False]
LS5_Training_=df1.sample(500)
LS5_Testing_=df1[df1.index.isin(LS5_Training.index)==False]
LS6_Training_=df1.sample(600)
LS6_Testing_=df1[df1.index.isin(LS6_Training.index)==False]
LS7_Training_=df1.sample(700)
LS7_Testing_=df1[df1.index.isin(LS7_Training.index)==False]
LS8_Training_=df1.sample(800)
LS8_Testing_=df1[df1.index.isin(LS8_Training.index)==False]
LS9_Training_=df1.sample(900)
LS9_Testing_=df1[df1.index.isin(LS9_Training.index)==False]

In [68]:
#Nine different learning schemes in the second normalization method.
LS1_Training=df2.sample(100) #The training data contains 100 data points.
LS1_Testing=df2[df2.index.isin(LS1_Training.index)==False] #The validation data contains the other 900 data points.
LS2_Training=df2.sample(200)
LS2_Testing=df2[df2.index.isin(LS2_Training.index)==False]
LS3_Training=df2.sample(300)
LS3_Testing=df2[df2.index.isin(LS3_Training.index)==False]
LS4_Training=df2.sample(400)
LS4_Testing=df2[df2.index.isin(LS4_Training.index)==False]
LS5_Training=df2.sample(500)
LS5_Testing=df2[df2.index.isin(LS5_Training.index)==False]
LS6_Training=df2.sample(600)
LS6_Testing=df2[df2.index.isin(LS6_Training.index)==False]
LS7_Training=df2.sample(700)
LS7_Testing=df2[df2.index.isin(LS7_Training.index)==False]
LS8_Training=df2.sample(800)
LS8_Testing=df2[df2.index.isin(LS8_Training.index)==False]
LS9_Training=df2.sample(900)
LS9_Testing=df2[df2.index.isin(LS9_Training.index)==False]

## Define three neural network models and evaluation criterion 
In the paper, Adnan only chose neural network models with one hidden layer, reasoning too many hidden layer will increase the computational cost. One problem we might concern is wheather the model will be too simplified to provide a confident result under this choice. Same concerns go with his choice of solver and learning rate as well. We will check on these problems by defining different classifiers with different choice on solver and learning rate.

I listed belowing the parameters Adnan chose during his research. The first model will have a hidden model with 18 nodes, a learning rate of 0.0081, a momentum rate of 0.70. The second model will have a hidden model with 23 nodes, a learning rate of 0.0095, a momentum rate of 0.69. The third model will have a hidden model with 27 nodes, a learning rate of 0.0075, a momentum rate of 0.79.

We will also define another classifier with different choice on solver, learning_rate and hidden layer. Detailed parameter choice will be shown later.

To maintain consistency comparability with previous research, we will apply same evalutation criterion with Adnan. The criteria were firstly based on training the neural models under all schemes for a chosen number of iterations; 25,000 iterations were considered as sufficient, since a higher number of iterations would lead to increasing the computational costs. The second criterion was based on the network converging to the required error value of 0.008. Upon achieving either of these criteria, training the neural model is terminated and the time costs, in addition to the correct evaluation rates (accuracy rates) are recorded. 


In [69]:
#Three different Neural Network Medel in the paper
#we choose tollerence=0, so we can make sure iteration will not end until converged. 
clf11= MLPClassifier(activation='relu',solver='sgd',early_stopping=False,alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(18),momentum=0.7,learning_rate_init=0.0081,tol=0,random_state=3,learning_rate='adaptive')
clf12= MLPClassifier(activation='relu',early_stopping=False,solver='sgd',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(23),momentum=0.69,learning_rate_init=0.0095,tol=0,random_state=3,learning_rate='adaptive')
clf13= MLPClassifier(activation='relu',early_stopping=False,solver='sgd',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(27),momentum=0.79,learning_rate_init=0.0075,tol=0,random_state=3,learning_rate='adaptive')

In [70]:
#Three different Neural Network Medel with solver changed to adam
#we choose tollerence=0, so we can make sure iteration will not end until converged. 
clf21= MLPClassifier(activation='relu',solver='adam',early_stopping=False,alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(18),momentum=0.7,learning_rate_init=0.0081,tol=0,random_state=3,learning_rate='adaptive')
clf22= MLPClassifier(activation='relu',early_stopping=False,solver='adam',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(23),momentum=0.69,learning_rate_init=0.0095,tol=0,random_state=3,learning_rate='adaptive')
clf23= MLPClassifier(activation='relu',early_stopping=False,solver='adam',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(27),momentum=0.79,learning_rate_init=0.0075,tol=0,random_state=3,learning_rate='adaptive')

In [71]:
#Three different Neural Network Medel with lower learning_rate
#we choose tollerence=0, so we can make sure iteration will not end until converged. 
clf31= MLPClassifier(activation='relu',solver='sgd',early_stopping=False,alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(18),momentum=0.7,learning_rate_init=0.0035,tol=0,random_state=3,learning_rate='adaptive')
clf32= MLPClassifier(activation='relu',early_stopping=False,solver='sgd',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(23),momentum=0.69,learning_rate_init=0.0030,tol=0,random_state=3,learning_rate='adaptive')
clf33= MLPClassifier(activation='relu',early_stopping=False,solver='sgd',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(27),momentum=0.79,learning_rate_init=0.0025,tol=0,random_state=3,learning_rate='adaptive')

In [72]:
#Three different Neural Network Medel with two hidden layer
#we choose tollerence=0, so we can make sure iteration will not end until converged. 
clf41= MLPClassifier(activation='relu',solver='sgd',early_stopping=False,alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(18,10),momentum=0.7,learning_rate_init=0.0035,tol=0,random_state=3,learning_rate='adaptive')
clf42= MLPClassifier(activation='relu',early_stopping=False,solver='sgd',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(23,12),momentum=0.69,learning_rate_init=0.0030,tol=0,random_state=3,learning_rate='adaptive')
clf43= MLPClassifier(activation='relu',early_stopping=False,solver='sgd',alpha=1e-5,max_iter=25000,\
                    hidden_layer_sizes=(27,14),momentum=0.79,learning_rate_init=0.0025,tol=0,random_state=3,learning_rate='adaptive')

In [73]:
#define dataframe to store the scoring data for each fit, we will start from what paper did and try different solver
fit_result11=pd.DataFrame({'Learning Scheme':[],'Neural Network':[],'Loss':[],'iternum':[],\
                         'Training Time':[],'Predicting Time':[],'Testing Acc':[],'Training Acc':[],\
                         'Overall Acc':[]})
fit_result21=pd.DataFrame({'Learning Scheme':[],'Neural Network':[],'Loss':[],'iternum':[],\
                         'Training Time':[],'Predicting Time':[],'Testing Acc':[],'Training Acc':[],\
                         'Overall Acc':[]})
fit_result22=pd.DataFrame({'Learning Scheme':[],'Neural Network':[],'Loss':[],'iternum':[],\
                         'Training Time':[],'Predicting Time':[],'Testing Acc':[],'Training Acc':[],\
                         'Overall Acc':[]})
fit_result23=pd.DataFrame({'Learning Scheme':[],'Neural Network':[],'Loss':[],'iternum':[],\
                         'Training Time':[],'Predicting Time':[],'Testing Acc':[],'Training Acc':[],\
                         'Overall Acc':[]})
fit_result24=pd.DataFrame({'Learning Scheme':[],'Neural Network':[],'Loss':[],'iternum':[],\
                         'Training Time':[],'Predicting Time':[],'Testing Acc':[],'Training Acc':[],\
                         'Overall Acc':[]})

## Inplementation and experimental results
Now, we are going to train three neural network models using nine learning schemes, with a total of 27 fits.

We will record the iteration number, loss value, training time, predicting time(for each application), accuracy on traing dataset, accuracy on testing dataset and overal accuracy in the above dataframe for future evaluation.

In [74]:
#1st fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS1_Training_[['Attribute '+str(i) for i in range(1,25)]],LS1_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS1_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS1_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[0]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[0]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[0]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[0]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[0]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [75]:
#2nd fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS1_Training_[['Attribute '+str(i) for i in range(1,25)]],LS1_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS1_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS1_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[1]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[1]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[1]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[1]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[1]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [76]:
#3rd fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS1_Training_[['Attribute '+str(i) for i in range(1,25)]],LS1_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS1_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS1_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[2]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[2]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[2]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[2]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS1_Training[['Attribute '+str(i) for i in range(1,25)]],LS1_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS1_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS1_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS1_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[2]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [77]:
#4th fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS2_Training_[['Attribute '+str(i) for i in range(1,25)]],LS2_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS2_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS2_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[3]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[3]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[3]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[3]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[3]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [82]:
#5th fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS2_Training_[['Attribute '+str(i) for i in range(1,25)]],LS2_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS2_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS2_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[4]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[4]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[4]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[4]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[4]=insert1#insert the evaluation imformation into the dataframe







/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [84]:
#6th fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS2_Training_[['Attribute '+str(i) for i in range(1,25)]],LS2_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS2_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS2_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[5]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[5]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[5]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[5]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS2_Training[['Attribute '+str(i) for i in range(1,25)]],LS2_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS2_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS2_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS2_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS2_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS2_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS2','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[5]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [85]:
#7th fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS3_Training_[['Attribute '+str(i) for i in range(1,25)]],LS3_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS3_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS3_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[6]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[6]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[6]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[6]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[6]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [86]:
#8th fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS3_Training_[['Attribute '+str(i) for i in range(1,25)]],LS3_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS3_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS3_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[7]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[7]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[7]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[7]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[7]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [88]:
#9th fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS3_Training_[['Attribute '+str(i) for i in range(1,25)]],LS3_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS3_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS3_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[8]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[8]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[8]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[8]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS3_Training[['Attribute '+str(i) for i in range(1,25)]],LS3_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS3_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS3_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS3_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS3_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS3_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS3','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[8]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [89]:
#10th fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS4_Training_[['Attribute '+str(i) for i in range(1,25)]],LS4_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS4_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS4_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[9]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[9]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[9]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[9]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[9]=insert1#insert the evaluation imformation into the dataframe




In [90]:
#11th fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS4_Training_[['Attribute '+str(i) for i in range(1,25)]],LS4_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS4_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS4_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[10]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[10]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[10]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[10]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[10]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [91]:
#12th fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS4_Training_[['Attribute '+str(i) for i in range(1,25)]],LS4_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS4_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS4_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[11]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[11]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[11]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[11]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS4_Training[['Attribute '+str(i) for i in range(1,25)]],LS4_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS4_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS4_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS4_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS4_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS4_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS4','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[11]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [92]:
#13th fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS5_Training_[['Attribute '+str(i) for i in range(1,25)]],LS5_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS5_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS5_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[12]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[12]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[12]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[12]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[12]=insert1#insert the evaluation imformation into the dataframe




In [93]:
#14th fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS5_Training_[['Attribute '+str(i) for i in range(1,25)]],LS5_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS5_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS5_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[13]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[13]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[13]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[13]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[13]=insert1#insert the evaluation imformation into the dataframe




In [94]:
#15th fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS5_Training_[['Attribute '+str(i) for i in range(1,25)]],LS5_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS5_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS5_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[14]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[14]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[14]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[14]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS5_Training[['Attribute '+str(i) for i in range(1,25)]],LS5_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS5_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS5_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS5_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS5_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS5_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS5','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[14]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [96]:
#16th fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS6_Training_[['Attribute '+str(i) for i in range(1,25)]],LS6_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS6_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS6_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[15]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[15]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[15]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[15]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[15]=insert1#insert the evaluation imformation into the dataframe




/Users/sikaifeng/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [97]:
#17th fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS6_Training_[['Attribute '+str(i) for i in range(1,25)]],LS6_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS6_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS6_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[16]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[16]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[16]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[16]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[16]=insert1#insert the evaluation imformation into the dataframe




In [98]:
#18th fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS6_Training_[['Attribute '+str(i) for i in range(1,25)]],LS6_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS6_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS6_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[17]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[17]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[17]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[17]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS6_Training[['Attribute '+str(i) for i in range(1,25)]],LS6_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS6_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS6_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS6_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS6_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS6_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS6','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[17]=insert1#insert the evaluation imformation into the dataframe




In [99]:
#19th fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS7_Training_[['Attribute '+str(i) for i in range(1,25)]],LS7_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS7_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS7_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[18]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[18]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[18]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[18]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[18]=insert1#insert the evaluation imformation into the dataframe




In [101]:
#20th fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS7_Training_[['Attribute '+str(i) for i in range(1,25)]],LS7_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS7_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS7_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[19]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[19]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[19]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[19]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[19]=insert1#insert the evaluation imformation into the dataframe




In [102]:
#21st fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS7_Training_[['Attribute '+str(i) for i in range(1,25)]],LS7_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS7_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS7_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[20]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[20]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[20]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[20]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS7_Training[['Attribute '+str(i) for i in range(1,25)]],LS7_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS7_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS7_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS7_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS7_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS7_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS7','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[20]=insert1#insert the evaluation imformation into the dataframe




In [103]:
#22nd fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS8_Training_[['Attribute '+str(i) for i in range(1,25)]],LS8_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS8_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS8_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[21]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[21]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[21]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[21]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[21]=insert1#insert the evaluation imformation into the dataframe




In [104]:
#23rd fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS8_Training_[['Attribute '+str(i) for i in range(1,25)]],LS8_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS8_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS8_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[22]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[22]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[22]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[22]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[22]=insert1#insert the evaluation imformation into the dataframe




In [106]:
#24th fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS8_Training_[['Attribute '+str(i) for i in range(1,25)]],LS8_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS8_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS8_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[23]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[23]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[23]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[23]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS8_Training[['Attribute '+str(i) for i in range(1,25)]],LS8_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS8_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS8_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS8_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS8_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS8_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS8','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[23]=insert1#insert the evaluation imformation into the dataframe




In [107]:
#25th fit

#first set
tic=dt.now()#time before fitting
clf11.fit(LS9_Training_[['Attribute '+str(i) for i in range(1,25)]],LS9_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS9_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS9_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[24]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf11.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf11.n_iter_#return the iteration time during the fitting
currentloss=clf11.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf11.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf11.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf11.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[24]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf21.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf21.n_iter_#return the iteration time during the fitting
currentloss=clf21.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf21.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf21.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf21.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[24]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf31.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf31.n_iter_#return the iteration time during the fitting
currentloss=clf31.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf31.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf31.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf31.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[24]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf41.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf41.n_iter_#return the iteration time during the fitting
currentloss=clf41.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf41.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf41.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf41.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN1','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[24]=insert1#insert the evaluation imformation into the dataframe




In [108]:
#26th fit

#first set
tic=dt.now()#time before fitting
clf12.fit(LS9_Training_[['Attribute '+str(i) for i in range(1,25)]],LS9_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS9_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS9_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[25]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf12.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf12.n_iter_#return the iteration time during the fitting
currentloss=clf12.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf12.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf12.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf12.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[25]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf22.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf22.n_iter_#return the iteration time during the fitting
currentloss=clf22.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf22.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf22.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf22.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[25]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf32.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf32.n_iter_#return the iteration time during the fitting
currentloss=clf32.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf32.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf32.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf32.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[25]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf42.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf42.n_iter_#return the iteration time during the fitting
currentloss=clf42.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf42.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf42.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf42.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN2','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[25]=insert1#insert the evaluation imformation into the dataframe




In [109]:
#27th fit

#first set
tic=dt.now()#time before fitting
clf13.fit(LS9_Training_[['Attribute '+str(i) for i in range(1,25)]],LS9_Training_['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS9_Testing_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing_.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing_['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS9_Training_[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training_['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df1[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df1['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result11.loc[26]=insert1#insert the evaluation imformation into the dataframe

#second set
tic=dt.now()#time before fitting
clf13.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf13.n_iter_#return the iteration time during the fitting
currentloss=clf13.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf13.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf13.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf13.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result21.loc[26]=insert1#insert the evaluation imformation into the dataframe

#third set
tic=dt.now()#time before fitting
clf23.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf23.n_iter_#return the iteration time during the fitting
currentloss=clf23.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf23.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf23.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf23.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result22.loc[26]=insert1#insert the evaluation imformation into the dataframe


#fourth set
tic=dt.now()#time before fitting
clf33.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf33.n_iter_#return the iteration time during the fitting
currentloss=clf33.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf33.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf33.predict(LS9_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS9_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf33.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS9','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result23.loc[26]=insert1#insert the evaluation imformation into the dataframe


#fifth set
tic=dt.now()#time before fitting
clf43.fit(LS9_Training[['Attribute '+str(i) for i in range(1,25)]],LS9_Training['Target']) #fit the 1st ANN on LS1
toc=dt.now()#time after fitting
fitting_time=toc-tic#training time taken
iternum=clf43.n_iter_#return the iteration time during the fitting
currentloss=clf43.loss_#return the loss value of after fitting
tic=dt.now()#time before predicting
testing_pred=clf43.predict(LS9_Testing[['Attribute '+str(i) for i in range(1,25)]])#make prediction on testing dataset
toc=dt.now()#time after predicting
predicting_time=(toc-tic)/LS9_Testing.index.size#calculate average predicting time for each application
testing_acc=acc(LS9_Testing['Target'], testing_pred)#calculate the accuracy on testing dataset
training_pred=clf43.predict(LS1_Training[['Attribute '+str(i) for i in range(1,25)]])#make prediction on training dataset
training_acc=acc(LS1_Training['Target'], training_pred)#calculate the accuracy on training dataset
whole_pred=clf43.predict(df2[['Attribute '+str(i) for i in range(1,25)]])#make prediction on whole dataset
whole_acc=acc(df2['Target'], whole_pred)#calculate the accuracy on whole dataset
insert1={'Learning Scheme':'LS1','Neural Network':'ANN3','Loss':currentloss,'iternum':iternum,\
        'Training Time':fitting_time,'Predicting Time':predicting_time,'Testing Acc':testing_acc,\
        'Training Acc':training_acc,'Overall Acc':whole_acc}
fit_result24.loc[26]=insert1#insert the evaluation imformation into the dataframe




In [117]:
#show result for the table of Adnan model
fit_result11

,Learning Scheme,Loss,Neural Network,Overall Acc,Predicting Time,Testing Acc,Training Acc,Training Time,iternum
0,LS1,0.0124514,ANN1,0.738,0 days 00:00:00.000002,0.743333,1,0 days 00:00:06.763552,25000
1,LS1,0.0073564,ANN2,0.759,0 days 00:00:00.000001,0.757778,1,0 days 00:00:07.488981,25000
2,LS1,0.00693564,ANN3,0.749,0 days 00:00:00.000001,0.752222,1,0 days 00:00:07.014510,25000
3,LS2,0.0992601,ANN1,0.761,0 days 00:00:00.000002,0.75875,0.975,0 days 00:00:08.640148,23719
5,LS2,0.0313089,ANN3,0.78,0:00:00.000002,0.77125,1,0:00:09.478852,25000
6,LS3,0.432988,ANN1,0.764,0 days 00:00:00.000001,0.77,0.806667,0 days 00:00:00.771969,1281
7,LS3,0.41259,ANN2,0.757,0 days 00:00:00.000001,0.758571,0.813333,0 days 00:00:01.027145,1773
8,LS3,0.438007,ANN3,0.759,0:00:00.000002,0.761429,0.8,0:00:00.950076,1573
9,LS4,0.4849,ANN1,0.77,0 days 00:00:00.000002,0.778333,0.775,0 days 00:00:01.559076,2346
10,LS4,0.483353,ANN2,0.767,0 days 00:00:00.000002,0.771667,0.7825,0 days 00:00:01.729783,2667


In [118]:
#show result after using normal distribution as normalization
fit_result21

,Learning Scheme,Loss,Neural Network,Overall Acc,Predicting Time,Testing Acc,Training Acc,Training Time,iternum
0,LS1,0.00122365,ANN1,0.709,0 days 00:00:00.000001,0.676667,1,0 days 00:00:07.341532,25000
1,LS1,0.000821399,ANN2,0.701,0 days 00:00:00.000001,0.667778,1,0 days 00:00:07.315463,25000
2,LS1,0.000680129,ANN3,0.713,0 days 00:00:00.000001,0.681111,1,0 days 00:00:07.611068,25000
3,LS2,0.00236037,ANN1,0.779,0 days 00:00:00.000002,0.72375,1,0 days 00:00:10.074939,25000
5,LS2,0.00158011,ANN3,0.756,0 days 00:00:00.000001,0.695,1,0 days 00:00:10.504102,25000
6,LS3,0.00976668,ANN1,0.812,0 days 00:00:00.000002,0.731429,1,0 days 00:00:14.343821,25000
7,LS3,0.0104266,ANN2,0.795,0 days 00:00:00.000002,0.707143,1,0 days 00:00:09.074520,15307
8,LS3,0.00119821,ANN3,0.785,0 days 00:00:00.000002,0.692857,1,0 days 00:00:16.132437,25000
9,LS4,0.109279,ANN1,0.826,0 days 00:00:00.000002,0.728333,0.9725,0 days 00:00:04.900575,7576
10,LS4,0.0499555,ANN2,0.819,0 days 00:00:00.000002,0.698333,1,0 days 00:00:04.628748,6527


In [119]:
#show result after chaning the solver as 'adam'
fit_result22

,Learning Scheme,Loss,Neural Network,Overall Acc,Predicting Time,Testing Acc,Training Acc,Training Time,iternum
0,LS1,7.66472e-06,ANN1,0.696,0 days 00:00:00.000001,0.662222,1,0 days 00:00:05.139023,15291
1,LS1,7.49437e-06,ANN2,0.707,0 days 00:00:00.000001,0.674444,1,0 days 00:00:04.603303,13954
2,LS1,7.69552e-06,ANN3,0.711,0 days 00:00:00.000001,0.678889,1,0 days 00:00:05.590392,14737
3,LS2,7.96572e-06,ANN1,0.767,0 days 00:00:00.000001,0.70875,1,0 days 00:00:06.720570,15598
5,LS2,7.34808e-06,ANN3,0.757,0 days 00:00:00.000001,0.69625,1,0 days 00:00:07.016828,15926
6,LS3,2.0433e-05,ANN1,0.798,0 days 00:00:00.000002,0.711429,1,0 days 00:00:03.663006,5855
7,LS3,1.77897e-05,ANN2,0.797,0 days 00:00:00.000001,0.71,1,0 days 00:00:03.621789,5482
8,LS3,1.70597e-05,ANN3,0.781,0 days 00:00:00.000002,0.687143,1,0 days 00:00:04.654571,6384
9,LS4,2.65232e-05,ANN1,0.826,0 days 00:00:00.000002,0.71,1,0 days 00:00:03.267893,4639
10,LS4,1.96515e-05,ANN2,0.815,0 days 00:00:00.000002,0.691667,1,0 days 00:00:04.523995,5947


In [120]:
#show result after lower initial learning rate
fit_result23

,Learning Scheme,Loss,Neural Network,Overall Acc,Predicting Time,Testing Acc,Training Acc,Training Time,iternum
0,LS1,0.00420434,ANN1,0.71,0 days 00:00:00.000001,0.677778,1,0 days 00:00:06.925918,25000
1,LS1,0.00409249,ANN2,0.7,0 days 00:00:00.000001,0.666667,1,0 days 00:00:07.333010,25000
2,LS1,0.00300099,ANN3,0.714,0 days 00:00:00.000001,0.682222,1,0 days 00:00:07.891891,25000
3,LS2,0.00823917,ANN1,0.782,0 days 00:00:00.000001,0.7275,1,0 days 00:00:08.606805,25000
5,LS2,0.0081461,ANN3,0.757,0 days 00:00:00.000001,0.69625,1,0 days 00:00:09.668225,25000
6,LS3,0.0111168,ANN1,0.812,0 days 00:00:00.000001,0.731429,1,0 days 00:00:14.710593,25000
7,LS3,0.023358,ANN2,0.796,0 days 00:00:00.000002,0.708571,1,0 days 00:00:11.564083,18444
8,LS3,0.00592598,ANN3,0.786,0 days 00:00:00.000002,0.694286,1,0 days 00:00:17.329517,25000
9,LS4,0.110319,ANN1,0.824,0 days 00:00:00.000002,0.725,0.9725,0 days 00:00:11.440492,18425
10,LS4,0.0890006,ANN2,0.82,0 days 00:00:00.000002,0.711667,0.9825,0 days 00:00:15.152240,21265


In [121]:
#show result after applying two layer of hidden layer
fit_result24

,Learning Scheme,Loss,Neural Network,Overall Acc,Predicting Time,Testing Acc,Training Acc,Training Time,iternum
0,LS1,0.00572827,ANN1,0.696,0 days 00:00:00.000002,0.662222,1,0 days 00:00:09.642607,25000
1,LS1,0.00056732,ANN2,0.696,0 days 00:00:00.000002,0.662222,1,0 days 00:00:09.318848,25000
2,LS1,0.000486554,ANN3,0.698,0 days 00:00:00.000002,0.664444,1,0 days 00:00:10.913745,25000
3,LS2,0.00127412,ANN1,0.772,0 days 00:00:00.000002,0.715,1,0 days 00:00:12.417919,25000
5,LS2,0.000934551,ANN3,0.754,0 days 00:00:00.000002,0.6925,1,0 days 00:00:14.791023,25000
6,LS3,0.00737825,ANN1,0.779,0 days 00:00:00.000002,0.684286,1,0 days 00:00:14.914436,18975
7,LS3,0.000838638,ANN2,0.794,0 days 00:00:00.000002,0.705714,1,0 days 00:00:23.143065,25000
8,LS3,0.000560955,ANN3,0.774,0 days 00:00:00.000002,0.677143,1,0 days 00:00:24.851365,25000
9,LS4,0.0341018,ANN1,0.836,0 days 00:00:00.000002,0.728333,0.9975,0 days 00:00:09.091496,10410
10,LS4,0.0184466,ANN2,0.815,0 days 00:00:00.000002,0.693333,0.9975,0 days 00:00:27.261085,25000


# Conlusion

Looking above fit-result11 table, where we reproduce Adnan's paper, we compare it to the result obtained in Adnan's paper. We noticed there are evident difference between our results. Apparently, our work contains larger error when Learning Scheme is higher. The results indicated our best methods are all under LS4, 3 models produced largest predicting power.However, in the paper,the best methods indicated by paper is LS4/ANN2, LS5/ANN1, LS6/ANN3. The reason behind this situation is uncertain. But we may give our best guess. The paper published is quite short, and mainly talk about the idea behind every step we did here, instead of talking about detail of model setting. There are many parameters we should take into consideration, for example, alpha, batch_size, epsilon, etc. Minor difference in the choice of these parameter will result in an apparent different result.

From fit_result21, after we applied normal distribution as normalization method, we can find the iteration number is increasing, and the loss is decreasing. It makes sense, since our data contains more information that paper's normalization methods might cause lose. The best Learning Scheme is under LS6, where our model predict 70% accuracy for both ANN2 and ANN3.

From fit_result22, when we changed the solver to 'adam', we can find the iteration number drops, but the loss value decreases. The best methods are LS7, and ANN1, ANN2, ANN3 all have predicting accuracy in testing data set for about 75%.

From fit_result23, when we lowered learning rate, we can find the iteration number is increasing, but the error value is also increasing. The best methods for these models are also LS7, where 3 neural networks produces accuracy about 75%.

From fit_result24, when we applied two hidden layers in the model, we notice the error value decreses to a lower magnitude comparing to the previous methods. The predicting power does not change much, though. The best methods are under LS7 for ANN1, ANN2, ANN3, where all models have predicting accuracy of more than 70%.

To summarize above finding. The performance of neural network model highly depends on the choice of parameter. Different choice of parameter leads to the difference between our results and the paper's. Some of the paper's assumption is problematic. More hidden layer will decreases the error value of model. We should measure the gain and loss by applying a more complicated model comparing to the more computaional cost. But overall, our model has a good predicting power on the German Credit Data.